In [49]:
using POMDPs
using PyCall
using Printf
using StatsBase
using ProgressMeter
using Flux
using BenchmarkTools

include("footeAlphaTak.jl")

gym = pyimport("gym")
foote_tak = pyimport("foote_tak")
env = gym.make("Tak-v0")
env.reset()

(Dict{Any, Any}("obs" => Dict{Any, Any}("Player: " => 1, "Board: " => PyObject[PyObject [] PyObject [] PyObject []; PyObject [] PyObject [] PyObject []; PyObject [] PyObject [] PyObject []])), Dict{Any, Any}("info" => 0))

In [50]:
create_network()

Chain(
  Dense(91 => 64),                      # 5_888 parameters
  var"#41#44"(),
  Chain(
    SkipConnection(
      Chain(
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
      ),
    ),
    SkipConnection(
      Chain(
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
      ),
    ),
    SkipConnection(
      Chain(
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
        Conv((3,), 64 => 64, pad=1),    # 12_352 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
      ),
    ),

In [51]:
function get_move_probabilities(root::MCTSNode, env)
    valid_action_indices = [act[end] for act in env.get_valid_actions()]
    probabilities = zeros(length(env.actions))
    
    total_visit_count = sum([child.visit_count for child in root.children])
    
    for child in root.children
        if child.action[end] in valid_action_indices
            probabilities[child.action[end]] = child.visit_count / total_visit_count
        end
    end
    
    return probabilities
end

get_move_probabilities (generic function with 1 method)

In [52]:
##### Part 1: Self-play #####
function self_play(network, game_env, solver, num_games)
    data = []

    @showprogress for i in 1:num_games
        game_env.reset()
        game_history = []
        reward_collected = 0

        while !game_env.is_terminal()
            
            # Use MCTS to determine the move probabilities
            node = solve(solver, game_env, network)
            
            move_probabilities = get_move_probabilities(node, game_env)

            # Store the state, move probabilities, and current player
            push!(game_history, (game_env.state, move_probabilities))
            
            action_index = argmax(get_move_probabilities(node, game_env))

            # Make a move
            _, reward_collected, _, _ = game_env.step(action_index)
        end

        # Determine the winner
        #winner = get_winner(game_env, reward_collected)

        # Add the game history and outcome to the training data
        for move in game_history
            push!(data, (move, reward_collected))
        end
    end

    return data
end

self_play (generic function with 1 method)